In [136]:
import pickle
import json
import numpy as np
from operator import itemgetter
from nltk.tokenize import word_tokenize
import re
import string

In [137]:
# drones
# tfhub_data_path = '/home/olga/Documents/drones_paragraph_ranking.pkl'
# rnet_data_path = '/home/olga/Documents/r_net_ranker_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/ODQA/data/PMEF/ranker1_drones_all_paragraphs.json'

#google
# tfhub_data_path = '/home/olga/Documents/wiki_chunks_tfhub.pkl'
# rnet_data_path = '/home/olga/Documents/wiki_r_net_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_google.json'

#git
# tfhub_data_path = '/home/olga/Documents/git_paragraph_ranking.pkl'
# rnet_data_path = '/home/olga/Documents/git_r_net_scores.pckl'
# tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_git.json'

# drones #2
tfhub_data_path = '/home/olga/Documents/drone_paragraph_ranking.pkl'
rnet_data_path = '/home/olga/Documents/drones_r_net_scores.pckl'
tfidf_data_path = '/media/olga/Data/projects/iPavlov/DeepPavlov/download/pmef/ensemble_drones.json'

In [138]:
def load_pickle(path):
    with open(path, 'rb') as fin:
        return pickle.load(fin)

In [139]:
def load_json(path):
    with open(path) as fin:
        return json.load(fin)

In [140]:
tfhub_data = load_pickle(tfhub_data_path)
rnet_data = load_pickle(rnet_data_path)
tfidf_data = load_json(tfidf_data_path)

In [141]:
len(tfhub_data)

149

In [142]:
len(rnet_data)

149

In [143]:
len(tfidf_data)

149

In [144]:
len(tfidf_data[130][2])

7

In [145]:
tfhub_data[0]

('what country manufactured the helicopter?',
 'Russia',
 [('This occurred on the first launch as the wind was not in Austria\'s favor on July 12, 1849 The Austrians had been developing this system for 66 years, since the idea was hatched in 1783. The Presse, of Vienna, Austria, stated: "Venice is to be bombarded by balloons, as the lagunes prevent the approaching of artillery. Five balloons, each measuring 5.7 meters in diameter, are in construction at Treviso. In a favorable wind the balloons will be launched and directed as near to Venice as possible, and on their being brought to vertical positions over the town, they will be fired by electro magnetism by means of a long isolated copper wire with a large galvanic battery placed on a building. The bomb falls perpendicularly, and explodes on reaching the ground." Balloons do not generally meet today\'s definition of a UAV. Once winged aircraft had been invented, the effort to fly them unmanned for military purposes was not far behind

In [146]:
rnet_data[0]

('what country manufactured the helicopter?',
 'Russia',
 [(0.9998914003372192,
   'Sberbank accounts for 28.9% of the total banking assets of the country (as of January 1, 2018), and its active customers are 60% of the population. Sberbank is among the top 5 largest employers of Russia. The Bank is the main creditor of the Russian economy and holds the largest share in the deposit market. It accounts for 46.1% of household deposits, 40.5% of loans to individuals and 32.4% of loans to legal entities (as of January 1, 2018). Sberbank consists of 14 territorial banks and 14.2 thousand branches in the Russian Federation. The Bank\'s foreign network consists of subsidiaries, branches and representative offices in the CIS, Central and Eastern Europe, Turkey, the United Kingdom, the United States and other countries. The number of active retail customers of Sberbank in Russia exceeds 86 million people, the number of corporate clients of the Group is more than 2 million in Russia. The range o

In [147]:
for item in tfidf_data:
    print(item[0])

what country manufactured the helicopter?
in which year was the helicopter created?
weight of a drones?
how long does the helicopter work?
What are the dimensions of this model of unmanned helicopter?
where you can buy?
where you can buy a helicopter?
where is the helicopter?
where is it?
What is the carrying capacity of a helicopter?
What is the weight of the helicopter?
What is the carrying capacity of a helicopter?
What is the mass of the helicopter?
What time can be in the air?
what amount of payload is it capable of carrying?
what is the weight of the helicopter?
how long can the helicopter be in the air?
how long is the helicopter capable of being in the air?
how long is the duration of a helicopter flight?
how much does the helicopter weigh?
how long can a it stay in the air without moving around ?
how many types of unmanned helicopters are there?
how many types does it have?
how many wings it has?
what is the average weight of an unmanned helicopter?
what is the benefit of it?


In [148]:
# make paragraph: id map

all_pars = []
for instance in rnet_data:
    pars = instance[2]
    all_pars += [p[1] for p in pars]

all_pars = list(set(all_pars))

In [149]:
len(all_pars)

7

In [150]:
par2id = {all_pars[i]: i for i in range(len(all_pars))}

In [151]:
id2par = {v: k for k, v in par2id.items()}

In [152]:
all_pars_tfhub = []
for instance in tfhub_data:
    pars = instance[2]
    all_pars_tfhub += [p[0] for p in pars]

all_pars_tfhub = list(set(all_pars_tfhub))

for p in all_pars_tfhub:
    if p not in par2id.keys():
        print(p)
else:
    print('All pars exist.')

All pars exist.


In [153]:
all_pars_tfidf = []
for instance in tfidf_data:
    pars = instance[2]
    all_pars_tfidf += [p[0] for p in pars]

all_pars_tfidf = list(set(all_pars_tfidf))

for p in all_pars_tfidf:
    if p not in par2id.keys():
        print(p)
        print('\n')
else:
    print('All pars exist.')

All pars exist.


In [154]:
all_questions = [item[0] for item in rnet_data]

In [155]:
all_questions

['what country manufactured the helicopter?',
 'in which year was the helicopter created?',
 'weight of a drones?',
 'how long does the helicopter work?',
 'What are the dimensions of this model of unmanned helicopter?',
 'where you can buy?',
 'where you can buy a helicopter?',
 'where is the helicopter?',
 'where is it?',
 'What is the carrying capacity of a helicopter?',
 'What is the weight of the helicopter?',
 'What is the carrying capacity of a helicopter?',
 'What is the mass of the helicopter?',
 'What time can be in the air?',
 'what amount of payload is it capable of carrying?',
 'what is the weight of the helicopter?',
 'how long can the helicopter be in the air?',
 'how long is the helicopter capable of being in the air?',
 'how long is the duration of a helicopter flight?',
 'how much does the helicopter weigh?',
 'how long can a it stay in the air without moving around ?',
 'how many types of unmanned helicopters are there?',
 'how many types does it have?',
 'how many w

In [156]:
# modify anaphopric questions

# anaphora_tokens = ['it', 'this', 'these', 'they']
# new_questions = []
# prev_tokens = []
# for i in range(len(all_questions)):
#     q = all_questions[i]
#     cur_tokens = word_tokenize(q)
#     if (any(t in cur_tokens for t in anaphora_tokens)):
#         if prev_tokens:
#             new_question = ' '.join(set(cur_tokens + prev_tokens))
#             print(new_question)
#             new_questions.append(new_question)
#         else:
#             new_questions.append(all_questions[i])
#     else:
#         new_questions.append(all_questions[i])
#         prev_tokens = cur_tokens
    
    

In [157]:
# len(new_questions)

In [158]:
all_answers = [item[1] for item in rnet_data]

In [159]:

mean_rankers_data = []

for q, a in zip(all_questions, all_answers):
#     print(q)
    tfhub_par_scores = [item[2] for item in tfhub_data if item[0]==q][0]
    tfhub_par_id_scores = {par2id[pair[0]]: pair[1] for pair in tfhub_par_scores}
    rnet_par_scores = [item[2] for item in rnet_data if item[0]==q][0]
    rnet_par_id_scores = {par2id[pair[1]]: pair[0] for pair in rnet_par_scores}
#     tfidf_par_scores = []
#     for item in tfidf_data:
#         if item[0] == q:
#             tfidf_par_scores.append(item[2])
#     print(type(tfidf_par_scores))
#     print(len(tfidf_par_scores))
#     tfidf_par_scores = tfidf_par_scores[0]
    try:
        tfidf_par_scores = [item[2] for item in tfidf_data if item[0]==q][0]
    except IndexError:
        print(q, a)
    tfidf_par_id_scores = {par2id[pair[0]]: pair[1] for pair in tfidf_par_scores}
    
    norm = np.linalg.norm(list(tfidf_par_id_scores.values()))
    tfidf_par_id_scores = {k: v / norm for k, v in tfidf_par_id_scores.items()}
    
#     print(tfhub_par_id_scores)
#     print(rnet_par_id_scores)
#     print(tfidf_par_id_scores)
    
    idx2mean = []
    for idx in par2id.values():
        tfhub_score = tfhub_par_id_scores[idx]
        rnet_score = rnet_par_id_scores[idx]
        tfidf_score = tfidf_par_id_scores[idx]
        
        mean_score = np.mean([tfhub_score, rnet_score, tfidf_score])
        idx2mean.append((idx, mean_score))
        
    idx2mean.sort(key=itemgetter(1))
    idx2mean = idx2mean[::-1]
    
    mean_rankers_data.append((q, a, idx2mean))
    

In [160]:
id2par[2]

'It consists of more than 700 organizations, of which currently there are 9 holding companies in the defense industrial complex and 6 in the civilian industries, as well as 32 direct management organizations. Rostekh\'s portfolio includes such well-known brands as AVTOVAZ, KAMAZ, Kalashnikov Concern, "Russian Helicopters", VSMPO-AVISMA, etc. The organizations of Rostech are located in the territory of 60 subjects of the Russian Federation and supply products to the markets of more than 70 countries. Consolidated revenue of Rostech in 2016 reached 1 trillion 266 billion rubles, consolidated net profit - 88 billion rubles, and EBITDA - 268 billion rubles. The average salary for the Corporation in 2016 was 44,000 rubles. According to Rostekh\'s new strategy, the main goal of the Corporation is to ensure Russia\'s technological advantage in highly competitive global markets. One of Rostekh\'s key tasks is the introduction of a new technological structure and the digitalization of the Russi

In [161]:
mean_rankers_data[0]

('what country manufactured the helicopter?',
 'Russia',
 [(1, 0.75124718045231109),
  (6, 0.46900785590286209),
  (3, 0.36306148407340483),
  (5, 0.30725516490807009),
  (2, 0.29254757615208421),
  (0, 0.26609178024014268),
  (4, 0.14120617148757411)])

In [162]:
def instance_score1(answer, texts):
    for doc_text in texts:
        if doc_text.find(answer) != -1:
            return 1
    return 0

def instance_score(a, texts):
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    a = regex.sub(' ', a)
    a = re.sub(" +", " ", a)
    a = a.replace('\n', ' ')
    for doc_text in texts:
        doc_text = regex.sub(' ', doc_text)
        doc_text = re.sub(" +", " ", doc_text)
        doc_text = doc_text.replace('\n', ' ')
        if doc_text.find(a.strip()) != -1:
            return 1
    return 0

In [163]:
recall_mapping = {}
for n in range(1, len(id2par) +1):
    correct_answers = 0
    for instance in mean_rankers_data:
        top_n_ids = [pair[0] for pair in instance[2]][:n]
        top_n_texts = [id2par[j] for j in top_n_ids]
        top_n_texts = [text.lower() for text in top_n_texts]
        formatted_answer = instance[1].strip().lower()
        correct_answers += instance_score(formatted_answer, top_n_texts)
    recall = correct_answers / len(rnet_data)
    recall_mapping[n] = recall

In [164]:
recall_mapping

{1: 0.4966442953020134,
 2: 0.6778523489932886,
 3: 0.8926174496644296,
 4: 0.9731543624161074,
 5: 0.9865771812080537,
 6: 0.9932885906040269,
 7: 1.0}

In [209]:
len(rnet_data)

40